In [278]:
import pandas as pd
import torch
from torch import nn
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
from torchvision import datapoints as dp
    
torch.manual_seed(17)



# img = read_image('/Users/viktor/polsl/bachelor_project/DatasetTraffic/dataset/sec1/sec1_frame_004500.png')
# bbox = [[372.94944000000004, 384.88014, 412.82976, 400.91058],[377.38944, 395.14985999999993, 415.40928, 411.62958], [123.80928, 554.7997799999999, 183.27936, 586.34982], [587.55072, 201.01014, 600.0, 208.75050000000002]]
# bbox = torch.tensor(bbox, dtype=torch.int)
# print(bbox)
# print(bbox.size())

# img=draw_bounding_boxes(img, bbox, width=3, colors=[(255,0,0),(0,255,0), (0,0,23), (25,25,25)])
# img = torchvision.transforms.ToPILImage()(img)
# img.show()
def random_RGB_colors(num):
    return [tuple(np.random.randint(1, 256, size=3)) for i in range(num)]


In [296]:
class TrafficDataset(torch.utils.data.Dataset):
    def __init__(self, root_path, transforms=None):
        self.root_path = root_path
        self.transforms = transforms

        self.frames, self.labels = self.__get_frames_labels(self.root_path)
    
    
    def __get_frames_labels(self, root_path):
        frames = []
        labels = []
        for _, _, files in os.walk(self.root_path):
            sorted_files = sorted(files)
            for id, file in enumerate(sorted_files[1::2]):
                labels.append(os.path.join(sec1, file))
                frames.append(os.path.join(sec1, sorted_files[id*2]))
        return tuple(frames), tuple(labels)

    def __getitem__(self, idx):
        frame = dp.Image(read_image(self.frames[idx]))
        label_txt = self.labels[idx]
        

        boxes = []
        labels = []
        with open(label_txt) as f:
            for vehicle in f.readlines():
                vehicle_type, top_left_x, top_left_y, bottom_right_x, bottom_right_y = vehicle.split()
                # xmin, ymin, xmax, ymax
                boxes.append([float(top_left_x), float(top_left_y), float(bottom_right_x), float(bottom_right_y)])
                labels.append(int(vehicle_type))
        boxes = torch.tensor(boxes, dtype=torch.float32)
        print(boxes, labels)
        target = {}
        target["boxes"] = boxes  
        target["label"] = labels

        if self.transforms is not None:
            frame, target = self.transforms(frame, target)
        return frame, target

    def __len__(self):
        return len(self.frames)

        



current_path = os.path.dirname(os.path.abspath("__file__"))
dataset_path = os.path.join(current_path, "..", "DatasetTraffic", "dataset", "sec1")

obj = TrafficDataset(dataset_path)

frame, target = obj[100]
img = draw_bounding_boxes(frame, target["boxes"], width=3, colors=random_RGB_colors(num=len(target["label"])))
img = torchvision.transforms.ToPILImage()(img)
img.show()




tensor([[1522.0695,  312.4100, 1544.4701,  338.6400],
        [1594.5110,  287.3399, 1613.9203,  314.2697],
        [1676.0900,  275.0495, 1696.4707,  298.8300],
        [1730.0496,  248.5204, 1747.1587,  269.0501],
        [1773.1210,  236.2797, 1787.9607,  258.0298],
        [1841.9404,  194.3703, 1860.3110,  209.5303]]) [1, 1, 1, 1, 1, 0]
